# Football Scheduler

Here we implement the model descripted at [[1]](#References) and run some test with real teams.

## Imports

In [22]:
import json
import random
import pandas as pd
from typing import Callable, Tuple, List
from IPython.display import display, Markdown

from model import FootballSchedulerModel, SymetricScheme
from parse import parse_sol, to_df

In [2]:
countries = ["ARG", "BOL", "BRA", "CHI", "COL", "ECU", "PAR", "PER", "URU", "VEN"]
top_countries = ["ARG", "BRA"]
assert(set(top_countries).issubset(set(countries)))
n = len(countries)

In [19]:
def raffle(countries: list[str]) -> Tuple[Callable[[str], int], Callable[[int], str]]:
	countries_copy = countries.copy()
	random.shuffle(countries_copy)
	return lambda c: countries_copy.index(c), lambda i: countries_copy[i]

def write_raffle(base_path: str, index_of: Callable[[str], int], country_of: Callable[[int], str]):
	with open(f"{base_path}/index_of.json", "w") as f:
		json.dump(dict([(c, index_of(c)) for c in countries]), f)
	with open(f"{base_path}/country_of.json", "w") as f:
		json.dump(dict([(i, country_of(i)) for i in range(n)]), f)

# Represents the random asignment of country <-> id.
index_of, country_of = raffle(countries)

In [20]:
top_teams = [index_of(c) for c in top_countries]

In [33]:
def write_model_and_raffle(
    base_path: str, model: FootballSchedulerModel
):
    write_raffle(base_path, index_of=index_of, country_of=country_of)
    model.write_problem(f"{base_path}/model.lp")


def assert_optimized(model: FootballSchedulerModel):
	model.optimize()
	assert model.get_obj_value() == 0

def write_solution(base_path: str, model: FootballSchedulerModel):
	sol_path = f"{base_path}/solution.sol"
	model.write_sol(sol_path)

def recover_solution_as_df(base_path:str) -> pd.DataFrame:
	solution_path = f"{base_path}/solution.sol"
	index_of_path = f"{base_path}/index_of.json"
	country_of_path = f"{base_path}/country_of.json"

	sol = parse_sol(solution_path)
	index_of = json.load(open(index_of_path))
	country_of = json.load(open(country_of_path))

	return to_df(
		sol=sol,
		index_of=index_of,
		country_of=country_of,
	)

def write_solution_to_latex(base_path: str, sol_df: pd.DataFrame):
	with open(f"{base_path}/solution_table.tex", "w+") as f:
		f.write(sol_df.to_latex(index=False))

## French Scheme

In [35]:
base_path_french = "output/french"

In [36]:
french_model = FootballSchedulerModel(n, SymetricScheme.FRENCH, top_teams)
write_model_and_raffle(base_path_french, french_model)
assert_optimized(french_model)
print(
    f"Optimization of the french model with top teams took: {french_model.get_solving_time()} seconds"
)
write_solution(base_path_french, french_model)

wrote problem to file /home/lgr/Desktop/UBA/Datos/2025/invop/tps/invop-football-scheduler/output/french/model.lp
Optimization of the french model with top teams took: 4.280376 seconds


In [37]:
french_df = recover_solution_as_df(base_path_french)
display(Markdown(french_df.to_markdown(index=False)))

| Team   | 0    | 1    | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   | 13   | 14   | 15   | 16   | 17   |
|:-------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| ARG    | ECU  | @BRA | @COL | VEN  | @CHI | ARG  | PAR  | @BOL | @URU | BRA  | COL  | @VEN | CHI  | @ARG | @PAR | BOL  | URU  | @ECU |
| BOL    | @VEN | CHI  | @BRA | COL  | @ARG | ECU  | BOL  | @PAR | PER  | @CHI | BRA  | @COL | ARG  | @ECU | @BOL | PAR  | @PER | VEN  |
| BRA    | PAR  | @ARG | @VEN | BRA  | @COL | CHI  | @URU | PER  | @ECU | ARG  | VEN  | @BRA | COL  | @CHI | URU  | @PER | ECU  | @PAR |
| CHI    | @CHI | BOL  | @PAR | ECU  | URU  | @PER | VEN  | @BRA | COL  | @BOL | PAR  | @ECU | @URU | PER  | @VEN | BRA  | @COL | CHI  |
| COL    | ARG  | @URU | @ECU | PAR  | PER  | @BOL | COL  | @VEN | @BRA | URU  | ECU  | @PAR | @PER | BOL  | @COL | VEN  | BRA  | @ARG |
| ECU    | @BOL | COL  | ARG  | @CHI | @ECU | BRA  | @PER | URU  | VEN  | @COL | @ARG | CHI  | ECU  | @BRA | PER  | @URU | @VEN | BOL  |
| PAR    | BRA  | @PAR | PER  | @URU | BOL  | @VEN | @CHI | ECU  | @ARG | PAR  | @PER | URU  | @BOL | VEN  | CHI  | @ECU | ARG  | @BRA |
| PER    | @COL | PER  | URU  | @BOL | VEN  | @PAR | @ECU | ARG  | CHI  | @PER | @URU | BOL  | @VEN | PAR  | ECU  | @ARG | @CHI | COL  |
| URU    | URU  | @ECU | BOL  | @PER | @BRA | COL  | @ARG | CHI  | @PAR | ECU  | @BOL | PER  | BRA  | @COL | ARG  | @CHI | PAR  | @URU |
| VEN    | @PER | VEN  | CHI  | @ARG | PAR  | @URU | BRA  | @COL | BOL  | @VEN | @CHI | ARG  | @PAR | URU  | @BRA | COL  | @BOL | PER  |

In [38]:
write_solution_to_latex(base_path_french, french_df)

## English Scheme

In [ ]:
base_path_french = "output/english"

## References:
- [1] G. Durán, E. Mijangos, and M. Frisk, “Scheduling the South American qualifiers to the 2018 FIFA World Cup by integer programming,” European Journal of Operational Research, vol. 262, no. 3, pp. 1035–1048, 2017.
- [2] Pyscipopt Docs.